___
<a href='https://honglab.ai'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

## 대형언어모델(LLM) 바닥부터 만들기

[유튜브 강의 영상 링크](https://youtu.be/osv2csoHVAo)

[홍정모 연구소 디스코드 링크](https://discord.com/invite/kgR9xJkbsV)

[홍정모 연구소 홈페이지 링크](https://www.honglab.ai/)

#### 참고 자료
- [Andrej Karpathy 유튜브](https://www.youtube.com/andrejkarpathy)
- [Build a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch)
- [Om-Alve/smolGPT 깃헙](https://github.com/Om-Alve/smolGPT)
- 트랜스포머 논문 - [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- OpenAI GPT2 논문 - [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)



#### 안내사항

LLM의 핵심 개념을 개인 PC에서도 간단하게 실습하면서 공부할 수 있는 학습 자료입니다. 널리 알려진 교육/학술 자료들을 참고하여 쉽게 공부할 수 있도록 요약하고 정리한 것입니다. 코딩 스타일이나 활용 범위에 대해 오해 없으시길 바랍니다.

윈도우11/WSL, Python 3.9.20, Pytorch 2.6, CUDA 12.6 에서 작동을 확인하였습니다. 

#### 전체 과정 요약

LLM 기반 AI 에이전트를 만들때는 핵심이 되는 LLM이 필요한데요, LLM을 바닥부터 만드는 경우 보다는 공개되어 있는 LLM 모델들을 가져다가 나의 용도에 맞도록 다듬어서 사용하는 것이 일반적입니다. 다만, 최근에는 LLM을 바닥부터 만드는 기술에 대한 진입장벽이 낮아지고 있어서 회사별로 필요한 LLM을 바닥부터 각자 만들어 사용하게 될 가능성도 높아지고 있습니다.

LLM을 만들 때는 

1. 사전훈련(pretraining)으로 일반적인 언어 능력을 가르친 후에 
2. 미세조정(fine tuning) 단계에서 특정 업무에 적응

시키는 것이 기본이 됩니다. 여기에 

3. 데이터베이스(+인터넷) 검색 기능을 추가

하면 지식의 범위와 정확성을 높일 수 있습니다. 사람이 생각을 거듭하여 더 깊이있는 결론을 이끌어 내듯이 LLM도 

4. 내부적으로 질의를 반복하여 더 좋은 결론을 도출

하도록 만들 수 있습니다.

여기서는 LLM의 기본 원리를 이해하기 위해서 사전훈련 과정을 바닥부터 진행해보겠습니다. 훈련 과정의 큰 틀은 일반적인 머신러닝 절차를 따릅니다.

1. 훈련 데이터 준비
1. 데이터 로더 정의
1. 모델 정의
1. 훈련
1. 결과 확인

#### 훈련 데이터 준비

준비한 텍스트 파일을 읽어 들여서 정리한 후에 앞에 cleaned_가 붙은 파일 이름으로 정리합니다.
> 예시) alice.txt &rarr; cleaned_alice.txt

- 캐글 해리포터 책 - [Harry Potter Books](https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books?select=02+Harry+Potter+and+the+Chamber+of+Secrets.txt)
- 캐글 앨리스 책 - [alice.txt](https://www.kaggle.com/datasets/leelatte/alicetxt)
- 훈련 데이터나 가중치는 제가 배포하지 않습니다. 직접 다운받거나 준비하셔야합니다.

In [1]:
import re

def clean_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        book_text = file.read()

    cleaned_text = re.sub(r'\n+', ' ', book_text) # 줄바꿈을 빈칸으로 변경
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # 여러 빈칸을 하나의 빈칸으로

    print("cleaned_" + filename, len(cleaned_text), "characters") # 글자 수 출력

    with open("cleaned_" + filename, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

filenames_list = ["02 Harry Potter and the Chamber of Secrets.txt"]

for filename in filenames_list:
    clean_text(filename)

cleaned_02 Harry Potter and the Chamber of Secrets.txt 488771 characters


#### 토큰화

UTF-8 BPE(Bype Pair Encoding)
- GPT-2
  - 서브워드 토크나이징: 자주 등장하지 않는 단어는 더 작은 단위로 쪼개짐
  - 효율성: 자주 등장하는 단어나 구는 하나의 토큰으로 표현됨
  - 다국어 처리: 영어가 아닌 언어는 토큰화가 덜 효율적일 수 있음

- 토큰화 된 숫자들은 GPT-2 토크나이저의 어휘 사전(vocabulary)에서 각 토큰에 할당된 고유 ID임.
- 예를 들어, "Harry"는 18308번, "Potter"는 14179번 같은 식으로 매핑되어 있음.

In [2]:
import tiktoken # pip install tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = "potter is the safest place to hide it.” Harry started to tell them about Colin, but Hermione interrupted. “We already know — we heard Professor McGonagall telling Professor Flitwick this morning. That’s why we decided we"

tokens = tokenizer.encode(text)

print("글자수:", len(text), "토큰수", len(tokens))
print(tokens)
print(tokenizer.decode(tokens))
for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")

글자수: 220 토큰수 52
[13059, 353, 318, 262, 33630, 1295, 284, 7808, 340, 13, 447, 251, 5850, 2067, 284, 1560, 606, 546, 18373, 11, 475, 19959, 19072, 13, 564, 250, 1135, 1541, 760, 851, 356, 2982, 8129, 11130, 261, 44906, 5149, 8129, 1610, 270, 16239, 428, 3329, 13, 1320, 447, 247, 82, 1521, 356, 3066, 356]
potter is the safest place to hide it.” Harry started to tell them about Colin, but Hermione interrupted. “We already know — we heard Professor McGonagall telling Professor Flitwick this morning. That’s why we decided we
13059	 -> pot
353	 -> ter
318	 ->  is
262	 ->  the
33630	 ->  safest
1295	 ->  place
284	 ->  to
7808	 ->  hide
340	 ->  it
13	 -> .
447	 -> �
251	 -> �
5850	 ->  Harry
2067	 ->  started
284	 ->  to
1560	 ->  tell
606	 ->  them
546	 ->  about
18373	 ->  Colin
11	 -> ,
475	 ->  but
19959	 ->  Hermione
19072	 ->  interrupted
13	 -> .
564	 ->  �
250	 -> �
1135	 -> We
1541	 ->  already
760	 ->  know
851	 ->  —
356	 ->  we
2982	 ->  heard
8129	 ->  Professor
11130	 ->  McG
26

In [ ]:
# from transformers import AutoTokenizer # pip install transformers

# tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")  # KoGPT2 사용
# # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")  # KoGPT2 사용

# print("Vocab size :", len(tokenizer))

# text = "대사께서는 도(道)를 얻은 모양이구려."

# tokens = tokenizer.encode(text)

# print(len(text), len(tokens))
# print(tokens)
# print(tokenizer.decode(tokens))

In [10]:
for char in text:
    token_ids = tokenizer.encode(char)     # 한 글자씩 인코딩(토큰화)
    decoded = tokenizer.decode(token_ids)  # 한 글자씩 디코딩
    print(f"{char} -> {token_ids} -> {decoded}")

p -> [79] -> p
o -> [78] -> o
t -> [83] -> t
t -> [83] -> t
e -> [68] -> e
r -> [81] -> r
  -> [220] ->  
i -> [72] -> i
s -> [82] -> s
  -> [220] ->  
t -> [83] -> t
h -> [71] -> h
e -> [68] -> e
  -> [220] ->  
s -> [82] -> s
a -> [64] -> a
f -> [69] -> f
e -> [68] -> e
s -> [82] -> s
t -> [83] -> t
  -> [220] ->  
p -> [79] -> p
l -> [75] -> l
a -> [64] -> a
c -> [66] -> c
e -> [68] -> e
  -> [220] ->  
t -> [83] -> t
o -> [78] -> o
  -> [220] ->  
h -> [71] -> h
i -> [72] -> i
d -> [67] -> d
e -> [68] -> e
  -> [220] ->  
i -> [72] -> i
t -> [83] -> t
. -> [13] -> .
” -> [447, 251] -> ”
  -> [220] ->  
H -> [39] -> H
a -> [64] -> a
r -> [81] -> r
r -> [81] -> r
y -> [88] -> y
  -> [220] ->  
s -> [82] -> s
t -> [83] -> t
a -> [64] -> a
r -> [81] -> r
t -> [83] -> t
e -> [68] -> e
d -> [67] -> d
  -> [220] ->  
t -> [83] -> t
o -> [78] -> o
  -> [220] ->  
t -> [83] -> t
e -> [68] -> e
l -> [75] -> l
l -> [75] -> l
  -> [220] ->  
t -> [83] -> t
h -> [71] -> h
e -> [68] -> e
m -> [7

#### 데이터로더(DataLoader)

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # token_ids = tokenizer.encode("<|endoftext|>" + txt, allowed_special={"<|endoftext|>"})
        token_ids = tokenizer.encode(txt)

        print("# of tokens in txt:", len(token_ids))

        for i in range(0, len(token_ids) - max_length, stride):
            # input_chunk는 모델에 입력되는 토큰 시퀀스야 (예: 토큰 0부터 max_length-1까지)
            # target_chunk는 모델이 예측해야 할 다음 토큰들이야 (예: 토큰 1부터 max_length까지)
            # stride 파라미터는 얼마나 겹치게 청크를 만들지 결정해 (작을수록 더 많은 겹침)
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# with open("cleaned_한글문서.txt", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
with open("cleaned_02 Harry Potter and the Chamber of Secrets.txt", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
    txt = file.read()

dataset = MyDataset(txt, max_length = 64, stride = 8)

train_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

print(tokenizer.n_vocab)

# 주의: 여기서는 코드를 단순화하기 위해 test, valid는 생략하고 train_loader만 만들었습니다.
#      관련된 ML 이론이 궁금하신 분들은 train vs test vs validation 등으로 검색해보세요.

# of tokens in txt: 130520
50257


In [14]:
dataiter = iter(train_loader)

x, y = next(dataiter)

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y[0].tolist()))


 said Harry. “Because whatever it is, it’s back and attacking people again —” His words were drowned by a loud outbreak of clicking and the rustling of many long legs shifting angrily; large black shapes shifted all around him. “The thing that lives in the castle,” said
 Harry. “Because whatever it is, it’s back and attacking people again —” His words were drowned by a loud outbreak of clicking and the rustling of many long legs shifting angrily; large black shapes shifted all around him. “The thing that lives in the castle,” said Ar


#### 뉴럴네트워크 모델 정의

모델 정의는 교재 "[Build a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch)"에서 제공하는 [예제 코드](https://github.com/rasbt/LLMs-from-scratch)를 약간 수정하였습니다.


NUM_LAYERS (레이어 수) :
트랜스포머 블록의 개수를 의미해
12개 → 6개로 줄이면: 모델의 깊이가 절반으로 줄어듦
영향: 모델이 학습할 수 있는 패턴의 복잡성이 감소하지만, 계산량과 메모리 사용량이 크게 줄어듦
비유: 12층 건물을 6층으로 줄이는 것과 같아. 더 적은 자원으로 건설할 수 있지만, 수용 가능한 사람(정보)이 줄어듦

EMB_DIM (임베딩 차원) :
모델 내부에서 각 토큰을 표현하는 벡터의 차원 수
768 → 384로 줄이면: 각 토큰의 표현력이 절반으로 줄어듦
영향: 토큰 간의 관계와 의미를 표현하는 능력이 감소하지만, 메모리 사용량과 계산량이 크게 줄어듦
비유: 사람을 표현할 때 768개의 특성(키, 몸무게, 성격 등)을 384개로 줄이는 것. 덜 세밀하지만 더 효율적

NUM_HEADS (어텐션 헤드 수) :
멀티헤드 어텐션에서 병렬로 수행되는 어텐션 계산의 수
12개 → 6개로 줄이면: 모델이 동시에 집중할 수 있는 다양한 패턴의 수가 줄어듦
영향: 다양한 관점에서 입력을 분석하는 능력이 감소하지만, 계산량이 줄어듦
비유: 12명의 전문가가 각각 다른 관점으로 문제를 분석하는 것을 6명으로 줄이는 것
이 세 가지 값을 줄이면 모델 크기와 복잡성이 크게 감소하여 학습 속도가 빨라지지만, 언어 이해 및 생성 능력은 감소해.

In [15]:
# 모델을 정의할 때 사용하는 상수들

VOCAB_SIZE = tokenizer.n_vocab # 50257 Tiktoken
#VOCAB_SIZE = len(tokenizer) # AutoTokenizer
CONTEXT_LENGTH = 64  # Shortened context length (orig: 1024)
EMB_DIM = 384  # Embedding dimension
NUM_HEADS = 6  # Number of attention heads
NUM_LAYERS = 6  # Number of layers
DROP_RATE = 0.1  # Dropout rate
QKV_BIAS = False  # Query-key-value bias
NUM_EPOCHS = 10

In [16]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        
        assert d_out % NUM_HEADS == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.head_dim = d_out // NUM_HEADS

        self.W_query = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_key = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_value = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(DROP_RATE)
        self.register_buffer('mask', torch.triu(torch.ones(CONTEXT_LENGTH, CONTEXT_LENGTH), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, NUM_HEADS, self.head_dim)
        values = values.view(b, num_tokens, NUM_HEADS, self.head_dim)
        queries = queries.view(b, num_tokens, NUM_HEADS, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(EMB_DIM, 4 * EMB_DIM),
            GELU(),
            nn.Linear(4 * EMB_DIM, EMB_DIM),
        )

    def forward(self, x):
        return self.layers(x)

class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=EMB_DIM,
            d_out=EMB_DIM)
    
        self.ff = FeedForward()
        self.norm1 = LayerNorm(EMB_DIM)
        self.norm2 = LayerNorm(EMB_DIM)
        self.drop_shortcut = nn.Dropout(DROP_RATE)

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.tok_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM)
        self.pos_emb = nn.Embedding(CONTEXT_LENGTH, EMB_DIM)
        self.drop_emb = nn.Dropout(DROP_RATE)

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock() for _ in range(NUM_LAYERS)])

        self.final_norm = LayerNorm(EMB_DIM)
        self.out_head = nn.Linear(EMB_DIM, VOCAB_SIZE, bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

#### 훈련

In [17]:
import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS 장치를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CPU를 사용합니다.")

print(device)

torch.manual_seed(123)
model = GPTModel()
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

MPS 장치를 사용합니다.
mps


In [22]:
tokens_seen, global_step = 0, -1

losses = []

for epoch in range(NUM_EPOCHS):
    model.train()  # Set model to training mode
    
    epoch_loss = 0
    for input_batch, target_batch in train_loader:
        optimizer.zero_grad() # Reset loss gradients from previous batch iteration
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)

        logits = model(input_batch)
        loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        epoch_loss += loss.item()
        loss.backward() # Calculate loss gradients
        optimizer.step() # Update model weights using loss gradients
        tokens_seen += input_batch.numel()
        global_step += 1

        if global_step % 100 == 0:
          # 기본 정보
          print(f"\n===== 학습 진행 상황 =====")
          print(f"🔄 에포크: {epoch + 1}/{NUM_EPOCHS}")
          print(f"📊 배치 크기: {input_batch.shape}")
          print(f"🔢 현재 배치 토큰 수: {input_batch.numel():,}개")
          print(f"📈 총 처리 토큰 수: {tokens_seen:,}개")
          print(f"📝 현재 손실값: {loss.item():.4f}")
          print(f"=======================\n")
        # Optional evaluation step

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss}")
    torch.save(model.state_dict(), "model_" + str(epoch + 1).zfill(3) + ".pth")

# 주의: 여기서는 편의상 모든 데이터를 train에 사용하였습니다. 
#      ML에서는 일부 데이터를 validation에 사용하는 것이 일반적입니다.


===== 학습 진행 상황 =====
🔄 에포크: 1/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 4,096개
📝 현재 손실값: 10.9948


===== 학습 진행 상황 =====
🔄 에포크: 1/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 413,696개
📝 현재 손실값: 5.0405


===== 학습 진행 상황 =====
🔄 에포크: 1/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 823,296개
📝 현재 손실값: 4.3614

Epoch: 1, Loss: 5.162279748541164

===== 학습 진행 상황 =====
🔄 에포크: 2/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 1,232,896개
📝 현재 손실값: 3.9236


===== 학습 진행 상황 =====
🔄 에포크: 2/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 1,642,496개
📝 현재 손실값: 3.6145


===== 학습 진행 상황 =====
🔄 에포크: 2/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 2,052,096개
📝 현재 손실값: 3.4294

Epoch: 2, Loss: 3.669385711977801

===== 학습 진행 상황 =====
🔄 에포크: 3/10
📊 배치 크기: torch.Size([64, 64])
🔢 현재 배치 토큰 수: 4,096개
📈 총 처리 토큰 수: 2,461,696개
📝 현재 손실값: 2.9687


===== 학습 진행 상황 =====
🔄 에포크: 3/10
📊 배치 크기: torch.S

In [18]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()

# 보충: validation loss를 같이 그려서 비교하는 사례 https://www.geeksforgeeks.org/training-and-validation-loss-in-deep-learning/

NameError: name 'losses' is not defined

#### 결과 확인

In [19]:
# 파일로 저장했던 네트워크의 가중치들 읽어들이기
model.load_state_dict(torch.load("model_010.pth", map_location=device, weights_only=True))
model.eval() # dropout을 사용하지 않음

GPTModel(
  (tok_emb): Embedding(50257, 384)
  (pos_emb): Embedding(64, 384)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=384, out_features=384, bias=False)
        (W_key): Linear(in_features=384, out_features=384, bias=False)
        (W_value): Linear(in_features=384, out_features=384, bias=False)
        (out_proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=

In [20]:
idx = tokenizer.encode("Dobby is") # 토큰 id의 list
idx = torch.tensor(idx).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(idx)

logits = logits[:, -1, :]

# 가장 확률이 높은 단어 10개 출력
top_logits, top_indices = torch.topk(logits, 10) 
for p, i in zip(top_logits.squeeze(0).tolist(), top_indices.squeeze(0).tolist()):
    print(f"{p:.2f}\t {i}\t {tokenizer.decode([i])}")

# 가장 확률이 높은 단어 출력
idx_next = torch.argmax(logits, dim=-1, keepdim=True)
flat = idx_next.squeeze(0) # 배치 차원 제거 torch.Size([1])
out = tokenizer.decode(flat.tolist()) # 텐서를 리스트로 바꿔서 디코드
print(out)

14.76	 991	  still
14.16	 1464	  always
12.97	 257	  a
12.07	 973	  used
11.22	 447	 �
11.10	 787	  make
11.09	 635	  also
11.02	 2156	  house
10.47	 262	  the
9.88	 1392	  got
 still


In [21]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        if top_k is not None:
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        if temperature > 0.0:
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:
            break

        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [22]:
start_context = input("Start context: ")

# idx = tokenizer.encode(start_context, allowed_special={'<|endoftext|>'})
idx = tokenizer.encode(start_context)
idx = torch.tensor(idx).unsqueeze(0)

context_size = model.pos_emb.weight.shape[0] 

for i in range(10):

    token_ids = generate(
        model=model,
        idx=idx.to(device),
        max_new_tokens=100,
        context_size= context_size,
        top_k=50,
        temperature=100
    )

    flat = token_ids.squeeze(0) # remove batch dimension
    out = tokenizer.decode(flat.tolist()).replace("\n", " ")

    print(i, ":", out)

0 : Potter is also more water…He gets you overtime away hopefully swiftlyaredings along so we used,.…powers fatal the gold stood frozen suddenly that coming ahead away this shining seriously because most popular sk C could possiblyaces into three floorest. When giving into sight that Ginny write also swinging to include prouffs on well than tu with much quite alone tightlytering close information because he sat where that eyes gave terror spiders appeared or she spoke into class people cheers without Fling bag by Mrs like him to They Hermione
1 : Potter is silence I Someone should school of Salohor when Fille toldet sold straight pleased quietly against Hogwarts fifty Trans without him up hair now? All 2 by Chamber years disappeared will— er— not eggted aggressively his faithfulENfect remained words Professor Behind Hogwarts he even thought savageiously brisks were perched emerging thrown with mud stern seemed in hot an old tapping Poking in his ribs and flooding next she raised anymor

#### 보충

- 여기서 소개해드린 LLM은 한 단어씩 만들어 가는 **자동회귀(autoregressive)** LLM 이라고 합니다. (자가회귀로 번역하기도 합니다.) 
- 최근에는 **디퓨전(Diffusion)** LLM 기술도 나오기 시작했습니다. 한번에 한 단어씩이 아니라 전체를 생성합니다. ([참고1](https://x.com/karpathy/status/1894923254864978091), [참고2](https://x.com/omarsar0/status/1891568386494300252))